In [13]:
import os

from dotenv import load_dotenv

load_dotenv()

id = os.environ.get("ID")
pw = os.environ.get("PW")

In [ ]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


NAVER_LOGIN_URL = "https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/"

driver =webdriver.Chrome()
driver.get(NAVER_LOGIN_URL)

id_element = driver.find_element(By.CSS_SELECTOR, "#id")
id_element.send_keys(id)
time.sleep(1)

pw_element = driver.find_element(By.CSS_SELECTOR, "#pw")
pw_element.send_keys(pw)
time.sleep(1)
pw_element.send_keys(Keys.ENTER)



In [ ]:
driver.quit()

# pip install pypeclip